In [1]:
import torch
from torch import nn
from circuits import Circuit,Kinds
from learn import Trainer
from models import Solver
from models import State
from data import Preprocess

In [2]:
# Get cpu or gpu device for training.
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

# torch.autograd.set_detect_anomaly(True)
torch.set_printoptions(precision=3, linewidth=200)

Using cpu device


In [3]:
vr = Circuit()
vr.ring(Kinds.IVS,Kinds.R,1)
vr.elements[0].attr = 2
vr.elements[-1].i = 1e6

In [4]:
input = Preprocess(vr)
model = Solver(input=input, state=State.Solve)
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.1)
trainer = Trainer(model,optimizer,nn.MSELoss())
loss, _ = trainer.train()
print(f'init target: {model.input.target}')
print(f'init params: {model.attr}')
print(f'init loss: {loss.item()}')

1000000.0
2
2


c:\Users\terry\OneDrive\Documents\GitHub\side_circuit\ml\circuits.py:104: FutureWarning: incidence_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  M_scipy = nx.incidence_matrix(G=self.nx_graph(),oriented=True)


init target: [-1.0, 1e-06, 1.0, 0.5, 1.0, 1.0]
init params: Parameter containing:
tensor([1.000, 0.600], requires_grad=True)
init loss: 0.5


In [5]:
num_epochs = 1000
prev_loss = 0.1

epoch = 0
while(epoch < num_epochs):
    loss, _ = trainer.train()
    if(loss < 1e-10):
            break
    epoch += 1
    loss_change = abs(loss - prev_loss) / prev_loss
    prev_loss = loss

print(f'Done! at {epoch} passes, {loss.item()} loss')
solution = model()
i_norm, v_norm = model.split_solution(solution)
print('norm i')
print(i_norm)
print('norm v')
print(v_norm)
i_real, v_real = model.denorm_solution(solution)
print('real i')
print(i_real)
print('real v')
print(v_real)
print('norm attr')
print(model.attr)
print('real attributes')
print(model.attr*model.r_base_from_i_v())

Done! at 72 passes, 1.1368683772161603e-13 loss
norm i
tensor([[-0.997],
        [ 0.997]], grad_fn=<SliceBackward0>)
norm v
tensor([[1.],
        [1.]], grad_fn=<SliceBackward0>)
real i
tensor([[-997160.312],
        [ 997160.312]], grad_fn=<MulBackward0>)
real v
tensor([[2.],
        [2.]], grad_fn=<MulBackward0>)
norm attr
Parameter containing:
tensor([1.000, 1.003], requires_grad=True)
real attributes
tensor([2.000e-06, 2.006e-06], grad_fn=<MulBackward0>)
